# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [43]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
import re
import pickle

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [44]:
# load data from database
engine = create_engine('sqlite:///../data/DisasterResponse.db')
df = pd.read_sql_table('disaster_messages', engine)

df = df[:200]

X = df['message']
Y = df.iloc[:, 4:]

In [45]:
df.head(3)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [47]:
Y.head(3)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 40 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      200 non-null    int64  
 1   message                 200 non-null    object 
 2   original                200 non-null    object 
 3   genre                   200 non-null    object 
 4   related                 200 non-null    float64
 5   request                 200 non-null    float64
 6   offer                   200 non-null    float64
 7   aid_related             200 non-null    float64
 8   medical_help            200 non-null    float64
 9   medical_products        200 non-null    float64
 10  search_and_rescue       200 non-null    float64
 11  security                200 non-null    float64
 12  military                200 non-null    float64
 13  child_alone             200 non-null    float64
 14  water                   200 non-null    fl

### 2. Write a tokenization function to process your text data

In [49]:
def tokenize(text):
    '''Function that takes in a text splits with white spaces and creates list of words'''
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())

    word_list = nltk.tokenize.word_tokenize(text)
    
    stop = stopwords.words("english")
    word_list = [t for t in word_list if t not in stop]

    lemmed_list = [WordNetLemmatizer().lemmatize(w) for w in word_list]
    
    return lemmed_list

In [50]:
test_text = 'Hello world. This is just a test'
tokenize(test_text)

['hello', 'world', 'test']

Works!

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [51]:
# Setting up the machine learning model
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
        ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [52]:
# Step one: Split into train- and test data
X_train, X_test, Y_train, Y_test = train_test_split(
            X, Y, test_size=0.2)

In [53]:
# Our model to train is "pipeline"
model = pipeline.fit(X_train, Y_train)

In [54]:
pred_train = model.predict(X_train)

In [55]:
pred_test = model.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [56]:
# print predictions classification_report()
for index, column in enumerate(Y_test.columns):
    print('Category: {}'.format(column))
    print(classification_report(Y_test[column], pred_test[:,index]))

Category: related
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         6
         1.0       0.85      1.00      0.92        34

    accuracy                           0.85        40
   macro avg       0.42      0.50      0.46        40
weighted avg       0.72      0.85      0.78        40

Category: request
              precision    recall  f1-score   support

         0.0       0.67      0.67      0.67        15
         1.0       0.80      0.80      0.80        25

    accuracy                           0.75        40
   macro avg       0.73      0.73      0.73        40
weighted avg       0.75      0.75      0.75        40

Category: offer
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        40

    accuracy                           1.00        40
   macro avg       1.00      1.00      1.00        40
weighted avg       1.00      1.00      1.00        40

Category: aid_related

In [57]:
# print predictions classification_report()
for index, column in enumerate(Y_train.columns):
    print('Category: {}'.format(column))
    print(classification_report(Y_train[column], pred_train[:,index]))

Category: related
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        23
         1.0       1.00      1.00      1.00       136
         2.0       1.00      1.00      1.00         1

    accuracy                           1.00       160
   macro avg       1.00      1.00      1.00       160
weighted avg       1.00      1.00      1.00       160

Category: request
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        66
         1.0       1.00      1.00      1.00        94

    accuracy                           1.00       160
   macro avg       1.00      1.00      1.00       160
weighted avg       1.00      1.00      1.00       160

Category: offer
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       159
         1.0       1.00      1.00      1.00         1

    accuracy                           1.00       160
   macro avg       1.0

### 6. Improve your model
Use grid search to find better parameters.

In [58]:
# What are the parameters of the pipeline object?
params = pipeline.get_params().keys()

In [59]:
for param in params:
    print(param + ': ', pipeline.get_params()[param], '\n')

memory:  None 

steps:  [('vect', CountVectorizer(tokenizer=<function tokenize at 0x000002C3FD548550>)), ('tfidf', TfidfTransformer()), ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))] 

verbose:  False 

vect:  CountVectorizer(tokenizer=<function tokenize at 0x000002C3FD548550>) 

tfidf:  TfidfTransformer() 

clf:  MultiOutputClassifier(estimator=RandomForestClassifier()) 

vect__analyzer:  word 

vect__binary:  False 

vect__decode_error:  strict 

vect__dtype:  <class 'numpy.int64'> 

vect__encoding:  utf-8 

vect__input:  content 

vect__lowercase:  True 

vect__max_df:  1.0 

vect__max_features:  None 

vect__min_df:  1 

vect__ngram_range:  (1, 1) 

vect__preprocessor:  None 

vect__stop_words:  None 

vect__strip_accents:  None 

vect__token_pattern:  (?u)\b\w\w+\b 

vect__tokenizer:  <function tokenize at 0x000002C3FD548550> 

vect__vocabulary:  None 

tfidf__norm:  l2 

tfidf__smooth_idf:  True 

tfidf__sublinear_tf:  False 

tfidf__use_idf:  True 

clf__est

#### Selecting some example parameters for the text pipeline:
- text_pipeline__vect__max_df:  1.0
- text_pipeline__vect__min_df:  1
- text_pipeline__vect__max_features:  None
- text_pipeline__vect__stop_words:  None
- text_pipeline__tfidf__use_idf:  True

#### Selecting some example parameters for the classifier pipeline:
- clf__estimator__max_depth:  None
- clf__estimator__min_samples_leaf:  1
- clf__estimator__n_estimators:  10

In [60]:
# Trying new parameters for grid search
parameters = {
        'tfidf__use_idf': (True, False),
        'clf__estimator__n_estimators': [10, 20]
}

cv = GridSearchCV(pipeline, param_grid = parameters)
cv

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x000002C3FD548550>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__n_estimators': [10, 20],
                         'tfidf__use_idf': (True, False)})

In [61]:
cv.fit(X_train, Y_train)

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x000002C3FD548550>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__n_estimators': [10, 20],
                         'tfidf__use_idf': (True, False)})

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [62]:
# Generating predictions
pred_test = cv.predict(X_test)
for index, column in enumerate(Y_train.columns):
    print('Category: {}'.format(column))
    print(classification_report(Y_test[column], pred_test[:,index]))

Category: related
              precision    recall  f1-score   support

         0.0       1.00      0.17      0.29         6
         1.0       0.87      1.00      0.93        34

    accuracy                           0.88        40
   macro avg       0.94      0.58      0.61        40
weighted avg       0.89      0.88      0.83        40

Category: request
              precision    recall  f1-score   support

         0.0       0.69      0.73      0.71        15
         1.0       0.83      0.80      0.82        25

    accuracy                           0.78        40
   macro avg       0.76      0.77      0.76        40
weighted avg       0.78      0.78      0.78        40

Category: offer
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        40

    accuracy                           1.00        40
   macro avg       1.00      1.00      1.00        40
weighted avg       1.00      1.00      1.00        40

Category: aid_related

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [63]:
file_name = 'classifier.pkl'
with open (file_name, 'wb') as dat_file:
    pickle.dump(cv, dat_file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.